#Теория
**Что за задачу мы хотим решить?**
Представим себе следующую задачу: мы хотим научиться с помощью нейросети генерировать объекты, неотличимые экспертом от реально существующих.

Для простоты рассмотрим конкретный пример: у нас дома сидит эксперт по кошкам, он знает про них всё. Нам нужно научиться генерировать изображения несуществовавших никогда кошек так, чтобы эксперт не смог отличить настоящие фотографии от сгенерированных.

На самом деле под кошками может скрываться все, что угодно. Любые изображения, которые нам взбредёт в голову генерировать, можно генерировать. Для этого существует спициальный алгоритм, который мы рассмотрим ниже:

**Как мы будем это делать?**
Generative adversarial networks (GAN), по-русски генеративно-состязательная сеть. Это модуль, состоящий из двух нейронных сетей: генератора и дескриминатора. 

Чтобы в конечном счёте обмануть настоящего эксперта, сначала создадим виртуального. Он будет представлять собой неронную сеть - дискриминатор. Задача этой нейронной сети - отличать настоящие объекты от сгенерированных. Будем учить её делать это хорошо

Генерировать ненастоящие объекты будет другая нейронная сеть - генератор. Её задача: научиться создавать изображения, которые дескриминатор посчитает настоящими.

Суть работы GAN в том, что две нейронные сети - генератор и дескриминатор, состязаются друг с другом, постоянно развиваясь. В конце концов генератор научится создавать изображения, неотличимые от оригинала, а дескриминатор - предельно точно распознавать подделки. Но задача прежде всего в том, чтобы научиться генерировать объекты, а не распознавать. Поэтому наш фокус внимания будет направлен на генератор.

# Models

In [ ]:
# импортируем зависимости
import numpy as np
from PuzzleLib.Backend import gpuarray
from PuzzleLib.Modules import Activation, sigmoid, relu, tanh, Linear, Reshape, BatchNorm1D
from PuzzleLib.Variable import Variable
from PuzzleLib.Containers import Graph, Sequential
import warnings


# блок генератора
def generator_block(in_feat, out_feat):
    # полносвязный слой
    linear = Linear(in_feat, out_feat, name="fc").node()
    # добавление размености
    rs = Reshape((1, out_feat, -1)).node(linear)
    # батчнорм слой
    bn = BatchNorm1D(out_feat, 0.8).node(rs)
    # активация
    act = Activation(relu, name="act").node(bn)
    # удаление размерности
    rs2 = Reshape((-1, out_feat)).node(act)
    return Graph(inputs=linear, outputs=rs2)


# генератор
def generator():
    layers = Sequential()
    layers.append(generator_block(32, 128))
    layers.append(generator_block(128, 256))
    layers.append(generator_block(256, 512))
    layers.append(generator_block(512, 1024))
    layers.append(Linear(1024, 32))
    layers.append(Activation(tanh))
    return layers

# дискриминатор
def discriminator():
    layers = Sequential()
    layers.append(Linear(32, 512))
    layers.append(Activation(relu))
    layers.append(Linear(512, 256))
    layers.append(Activation(relu))
    layers.append(Linear(256, 32))
    layers.append(Activation(sigmoid))
    return layers

In [ ]:
# инициализируем экземпляры дискриминатора и генератора
D = discriminator()
G = generator()

# Setup criterion and optimizer

In [ ]:
from PuzzleLib.Cost import BCE
from PuzzleLib.Optimizers import Adam

# ошибка
criterion = BCE()

# оптимизатор генератора
optimizer_g = Adam()
optimizer_g.setupOn(G, useGlobalState=True)

# оптимизатор дискриминатора
optimizer_d = Adam()
optimizer_d.setupOn(D, useGlobalState=True)

# Dataset

In [ ]:
from PuzzleLib.Datasets import Cifar10Loader

# создание датасета
cifar10 = Cifar10Loader()
path = "./TestData/"
data, _ = cifar10.load(path=path)

# Train

In [ ]:
for epoch in range(50):
    for i, img in enumerate(data):
        # генерируем Ground Truth ответы
        valid = gpuarray.to_gpu(np.ones((32, 32)).astype(np.int32))
        fake  = gpuarray.to_gpu(np.zeros((32, 32)).astype(np.int32))
        
        # -----------------
        #  Train Generator
        # -----------------
        
        # обнуляем градиенты
        optimizer_g.zeroGradParams()

        # генерируем шум
        z = gpuarray.to_gpu(np.random.normal(0, 1, (32, 32)).astype(np.float32))
        gen_imgs = G(z)
        
        # считаем ошибку и оптимизируем параметры
        g_loss, grad = criterion(D(gen_imgs).ravel(), valid.ravel())
        G.backward(grad.reshape(32, 32))
        optimizer_g.update()
        
        # ---------------------
        #  Train Discriminator
        # ---------------------
        
        # обнуляем градиенты
        optimizer_d.zeroGradParams()
        
        # считаем ошибку генератора на "сказать что настоящая картинка - настоящая"
        real_loss, real_grad = criterion(D(gpuarray.to_gpu(img[0, ...])).ravel(), valid.ravel())
        # считаем ошибку генератора на "сказать что фейковая картинка - фейковая"
        fake_loss, fake_grad = criterion(D(gen_imgs).ravel(), fake.ravel())
        # Общая ошибка - сумма ошибок в этих двух случаях
        d_loss = real_loss + fake_loss
        d_grad = real_grad + fake_grad
        
        
        # оптимизируем параметры дискриминатора
        D.backward(d_grad.reshape(32, 32))
        optimizer_d.update()
        
        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, 50, i, len(data), d_loss.item(), g_loss.item())
        )

[PuzzleLib] Warning: Module Reshape (name: 1) changed data batch axis size (was given (32, 128), reshaped to (1, 128, 32))
[PuzzleLib] Warning: Module Reshape (name: 4) changed data batch axis size (was given (1, 128, 32), reshaped to (32, 128))
[PuzzleLib] Warning: Module Reshape (name: 1) changed data batch axis size (was given (32, 256), reshaped to (1, 256, 32))
[PuzzleLib] Warning: Module Reshape (name: 4) changed data batch axis size (was given (1, 256, 32), reshaped to (32, 256))
[PuzzleLib] Warning: Module Reshape (name: 1) changed data batch axis size (was given (32, 512), reshaped to (1, 512, 32))
[PuzzleLib] Warning: Module Reshape (name: 4) changed data batch axis size (was given (1, 512, 32), reshaped to (32, 512))
[PuzzleLib] Warning: Module Reshape (name: 1) changed data batch axis size (was given (32, 1024), reshaped to (1, 1024, 32))
[PuzzleLib] Warning: Module Reshape (name: 4) changed data batch axis size (was given (1, 1024, 32), reshaped to (32, 1024))
[Epoch 0/50]